## 1. Load data from web page

Scrape the Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [234]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [235]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
postal_codes = tables[0]
postal_codes.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Filter dataframe deleting rows containing Not assigned Postal Codes. Rename first column.

In [236]:
postal_codes = postal_codes[postal_codes['Borough'] != 'Not assigned'].reset_index(drop=True)
postal_codes.rename(columns={'Postal code': 'PostalCode'}, inplace=True)

postal_codes.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In order to check duplicate postal codes we can count unique items for each column.

In [237]:
postal_codes.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,10,98
top,M9P,North York,Downsview
freq,1,24,4


No duplicate postal codes! It seems thet neighborhoods are already grouped by postal code.<br/>
But some neighborhoods are listed twice, since we have 98 unique neighborhoods over 103 postal codes. We will handle this issue later.<br/>
Let's check if there is any NaN or Not Assigned in Neighborhood column.

In [238]:
postal_codes['Neighborhood'].isna().sum()

0

In [239]:
(postal_codes['Neighborhood']=='Not assigned').sum()

0

Multiple Neighborhoods for the same Postal Code are formatted with '/', replace it with a single comma (,).

In [240]:
postal_codes['Neighborhood'].replace(' /',',',regex=True,inplace=True)
postal_codes.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [241]:
postal_codes.shape

(103, 3)

The first part of the project is complete.

## 2. Build Dataset

Let's add coordinates columns to the dataframe.

In [242]:
postal_codes['Latitude'] = 0.0
postal_codes['Longitude'] = 0.0

postal_codes.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,0.0,0.0
1,M4A,North York,Victoria Village,0.0,0.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0.0,0.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0.0,0.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0.0,0.0


Let's find out which neighborhoods appear more than once.

In [243]:
postal_codes_grouped = postal_codes.groupby('Neighborhood').count()

postal_codes_grouped[postal_codes_grouped['PostalCode']>1]

,PostalCode,Borough,Latitude,Longitude
Neighborhood,,,,
Don Mills,2,2,2,2
Downsview,4,4,4,4
Willowdale,2,2,2,2


In next dataframe iteration we will append Postal Code to these Neighborhoods in order to have unique Neighborhoods

Use geocoder to get coordinates and store them into dataframe. Use arcgis provider instead of google.

In [ ]:
! conda install -c conda-forge geocoder

import geocoder # import geocoder

print('geocoder imported!')

In [244]:
# iterate over dataframe and add values for latitude and longitude
for index, row in postal_codes.iterrows():
    postal_code = row['PostalCode']
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
    postal_codes.at[index,'Latitude']= g.lat
    postal_codes.at[index,'Longitude'] = g.lng
    if  row['Neighborhood'] in ['Don Mills','Downsview','Willowdale']:
        postal_codes.at[index,'Neighborhood'] = postal_codes.at[index,'Neighborhood'] + ' ' + postal_codes.at[index,'PostalCode']
    
# postal_codes contains all neighborhoods information now

Let's take a look at the dataframe now

In [245]:
postal_codes.shape

(103, 5)

In [246]:
postal_codes.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


Dataframe is now ready for further analysis!


## 3. Analyze Data

### 3.1 Explore Neighborhoods

Import libraries first.

In [ ]:
import json # library to handle JSON files

import requests # library to handle requests

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Get Toronto coordinates and create a map with neighborhoods

In [247]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817, -79.3839347.


In [249]:
# create map of Toronto using latitude and longitude values
map_toronto_full = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(postal_codes['Latitude'], postal_codes['Longitude'], postal_codes['Borough'], postal_codes['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_full)  
    
map_toronto_full

In [268]:
toronto_data = postal_codes.copy()
toronto_data.shape

(103, 5)

Foursquare Credentials

In [251]:
CLIENT_ID = 'QJ1PBI3IOFN5VJL5UCNGYC5NM5JVEXMUJP5VLXIE0V4VAT4S' #  Foursquare ID
CLIENT_SECRET = 'EVQVSSLLIO0RMLFEZMGBRZ2PTE4V2O2ZZ2JYDDYMOPZ5HCAP' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

credentials:
CLIENT_ID: QJ1PBI3IOFN5VJL5UCNGYC5NM5JVEXMUJP5VLXIE0V4VAT4S
CLIENT_SECRET:EVQVSSLLIO0RMLFEZMGBRZ2PTE4V2O2ZZ2JYDDYMOPZ5HCAP


Define Functions

In [252]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
  
#
# function that gets top 100 venues for each neighborhood
# names, latitudes, longitudes are the columns of the dataframe
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now let's build a new dataframe with all the venues

In [253]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])

Let's check the results

In [258]:
print(str(toronto_venues.shape[0]) + ' venues found in ' + str(toronto_venues.groupby('Neighborhood')['Neighborhood'].nunique().count()) + ' neighborhoods.')
print('Total number of neighborhoods in Toronto is ' + str(toronto_data.shape[0]))

2268 venues found in 101 neighborhoods.
Total number of neighborhoods in Toronto is 103


There are no nearby venues for 2 neighborhoods. Let's take a look at the dataframe.

In [259]:
grouped_toronto_venues = toronto_venues.groupby('Neighborhood').count().reset_index()
grouped_toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,5,5,5,5,5,5
1,"Alderwood, Long Branch",9,9,9,9,9,9
2,"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
3,Bayview Village,2,2,2,2,2,2
4,"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
5,Berczy Park,67,67,67,67,67,67
6,"Birch Cliff, Cliffside West",4,4,4,4,4,4
7,"Brockton, Parkdale Village, Exhibition Place",43,43,43,43,43,43
8,Business reply mail Processing CentrE,100,100,100,100,100,100
9,"CN Tower, King and Spadina, Railway Lands, Har...",66,66,66,66,66,66


And take a look at the categories

In [260]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


There are too few venues in some neighborhoods to perform a significant cluster analysis. I will exclude neighborhoods with less then 20 venues.

In [261]:
too_few = grouped_toronto_venues[grouped_toronto_venues['Venue'] < 20].reset_index()
print('There are ' + str(too_few.shape[0]) + ' neighborhoods with less than 20 nearby venues')

There are 68 neighborhoods with less than 20 nearby venues


In [262]:
indexes = toronto_venues[toronto_venues["Neighborhood"].isin(too_few['Neighborhood'])].index

toronto_venues_filtered = toronto_venues.drop(indexes)
toronto_venues_filtered.reset_index(inplace=True, drop=True)
toronto_venues_filtered.shape

(1888, 7)

In [283]:
toronto_venues_filtered.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.650964,-79.353041,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,"Regent Park, Harbourfront",43.650964,-79.353041,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue
2,"Regent Park, Harbourfront",43.650964,-79.353041,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
3,"Regent Park, Harbourfront",43.650964,-79.353041,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
4,"Regent Park, Harbourfront",43.650964,-79.353041,BATLgrounds,43.647088,-79.351306,Athletics & Sports


In [264]:
# do the same for toronto_data

toronto_data_filtered = toronto_data.copy()

indexes = toronto_data_filtered[toronto_data_filtered["Neighborhood"].isin(too_few['Neighborhood'])].index
toronto_data_filtered.drop(indexes , inplace=True)
toronto_data_filtered.reset_index(inplace=True, drop=True)
toronto_data_filtered.shape

#  there area still 2 neighborhoods for which the search returned no values...

(35, 5)

In [287]:
toronto_data_filtered.shape

(35, 5)

Now let's check if neighborhoods with too few venues have beeen correctly deleted

In [288]:
toronto_venues_filtered.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
Berczy Park,67,67,67,67,67,67
"Brockton, Parkdale Village, Exhibition Place",43,43,43,43,43,43
Business reply mail Processing CentrE,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",66,66,66,66,66,66
Canada Post Gateway Processing Centre,100,100,100,100,100,100
Central Bay Street,57,57,57,57,57,57
Church and Wellesley,79,79,79,79,79,79


Create a new map only with selected neighborhoods in new dataframe.
Same longitude and latitiude as before, a little more zoom.

In [271]:
# create map of Toronto using latitude and longitude values
map_toronto_filtered = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data_filtered['Latitude'], toronto_data_filtered['Longitude'], toronto_data_filtered['Borough'], toronto_data_filtered['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_filtered)  
    
map_toronto_filtered

Data preprocessing

In [289]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_filtered[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_filtered['Neighborhood'] 

toronto_onehot.shape

(1888, 227)

In [290]:
# move neighborhood column to the first column
nb = toronto_onehot.columns.get_loc('Neighborhood')
fixed_columns = [toronto_onehot.columns[nb]] + list(toronto_onehot.columns[: nb]) + list(toronto_onehot.columns[nb+1 :]) 

toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

And let's examine the new dataframe size.

In [291]:
toronto_onehot.shape

(1888, 227)

Calculate the mean of the occurrence of each category grouped by neighborhood

In [292]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.100000,0.000000,0.00000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000

In [293]:
toronto_grouped.shape

(33, 227)

Function for returning most common values

In [294]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [376]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Gas Station,Pet Store,Pizza Place,Deli / Bodega,Diner,Restaurant,Middle Eastern Restaurant
1,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Pharmacy,Thai Restaurant,Comfort Food Restaurant,Pizza Place,Pub,Restaurant,Café
2,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Italian Restaurant,Farmers Market,Bakery,Café,Breakfast Spot,Restaurant,Hotel
3,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Thrift / Vintage Store,Gift Shop,Diner,Sandwich Place,Caribbean Restaurant,Chiropractor,Restaurant,Cocktail Bar
4,Business reply mail Processing CentrE,Coffee Shop,Hotel,Café,American Restaurant,Asian Restaurant,Restaurant,Japanese Restaurant,Thai Restaurant,Steakhouse,Gym


### 3.2 Cluster Analysis

Run *k*-means to cluster the neighborhood into 5 clusters.

In [377]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 2, 0, 2, 0, 2, 0, 0, 0, 0, 2, 1, 3, 0, 0, 1, 2, 2, 3, 2, 2, 2,
       4, 2, 0, 2, 0, 2, 0, 2, 0, 2, 2], dtype=int32)

In [378]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data_filtered.drop('PostalCode',1)

# merge toronto_grouped with toronto_data_filtered to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,2.0,Pub,Music Venue,Coffee Shop,Café,Athletics & Sports,Bank,Bakery,Tech Startup,Thai Restaurant,Theater
1,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,3.0,Clothing Store,Toy / Game Store,American Restaurant,Furniture / Home Store,Men's Store,Bookstore,Restaurant,Cosmetics Shop,Women's Store,Coffee Shop
2,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,4.0,Coffee Shop,Café,Sushi Restaurant,Yoga Studio,Park,Spa,Burger Joint,Burrito Place,Sandwich Place,College Auditorium
3,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,0.0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Restaurant,Sandwich Place,Cosmetics Shop,Hotel,Bar,Café,Diner
4,Downtown Toronto,St. James Town,43.651734,-79.375554,0.0,Coffee Shop,Café,Italian Restaurant,Cosmetics Shop,American Restaurant,Gastropub,Hotel,Seafood Restaurant,Cocktail Bar,Department Store
5,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284,1.0,Fast Food Restaurant,Pizza Place,Thrift / Vintage Store,Fried Chicken Joint,Restaurant,Beer Store,Coffee Shop,Laundromat,Sports Bar,Breakfast Spot
6,Downtown Toronto,Berczy Park,43.645196,-79.373855,0.0,Coffee Shop,Seafood Restaurant,Cocktail Bar,Italian Restaurant,Farmers Market,Bakery,Café,Breakfast Spot,Restaurant,Hotel
7,East York,Leaside,43.709414,-79.363100,2.0,Coffee Shop,Sporting Goods Shop,Sports Bar,Burger Joint,Bank,Furniture / Home Store,Shopping Mall,Brewery,Breakfast Spot,Restaurant
8,Downtown Toronto,Central Bay Street,43.656072,-79.385653,0.0,Coffee Shop,Japanese Restaurant,Middle Eastern Restaurant,Café,Bubble Tea Shop,Clothing Store,Plaza,Breakfast Spot,Sandwich Place,Restaurant
9,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.757394,-79.442394,1.0,Coffee Shop,Bank,Pharmacy,Gas Station,Pet Store,Pizza Place,Deli / Bodega,Diner,Restaurant,Middle Eastern Restaurant


In [379]:
# the 2 neighborhoods with no venues are still here...

toronto_merged.dropna(inplace=True)

In [380]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

In [381]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [382]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Garden District, Ryerson",Coffee Shop,Clothing Store,Middle Eastern Restaurant,Restaurant,Sandwich Place,Cosmetics Shop,Hotel,Bar,Café,Diner
4,St. James Town,Coffee Shop,Café,Italian Restaurant,Cosmetics Shop,American Restaurant,Gastropub,Hotel,Seafood Restaurant,Cocktail Bar,Department Store
6,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Italian Restaurant,Farmers Market,Bakery,Café,Breakfast Spot,Restaurant,Hotel
8,Central Bay Street,Coffee Shop,Japanese Restaurant,Middle Eastern Restaurant,Café,Bubble Tea Shop,Clothing Store,Plaza,Breakfast Spot,Sandwich Place,Restaurant
10,"Richmond, Adelaide, King",Coffee Shop,Café,Clothing Store,Restaurant,Gym,American Restaurant,Deli / Bodega,Hotel,Salad Place,Sushi Restaurant
14,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,American Restaurant,Salad Place,Seafood Restaurant,Tea Room,Italian Restaurant
17,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Japanese Restaurant,Gym,Deli / Bodega,Beer Bar,Seafood Restaurant
23,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,Café,American Restaurant,Asian Restaurant,Restaurant,Japanese Restaurant,Thai Restaurant,Steakhouse,Gym
29,Stn A PO Boxes,Coffee Shop,Hotel,Café,American Restaurant,Asian Restaurant,Restaurant,Japanese Restaurant,Thai Restaurant,Steakhouse,Gym
32,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Gym,Japanese Restaurant,Asian Restaurant,Bar


#### Cluster 2

In [383]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,"Guildwood, Morningside, West Hill",Fast Food Restaurant,Pizza Place,Thrift / Vintage Store,Fried Chicken Joint,Restaurant,Beer Store,Coffee Shop,Laundromat,Sports Bar,Breakfast Spot
9,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Gas Station,Pet Store,Pizza Place,Deli / Bodega,Diner,Restaurant,Middle Eastern Restaurant
20,Downsview M3N,Shopping Mall,Grocery Store,Pizza Place,Discount Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint,Coffee Shop,Gas Station,Mobile Phone Shop


#### Cluster 3

In [384]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Pub,Music Venue,Coffee Shop,Café,Athletics & Sports,Bank,Bakery,Tech Startup,Thai Restaurant,Theater
7,Leaside,Coffee Shop,Sporting Goods Shop,Sports Bar,Burger Joint,Bank,Furniture / Home Store,Shopping Mall,Brewery,Breakfast Spot,Restaurant
13,"Little Portugal, Trinity",Restaurant,Cocktail Bar,Bar,Wine Bar,Vietnamese Restaurant,Asian Restaurant,Coffee Shop,Yoga Studio,Cupcake Shop,Brewery
15,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Thrift / Vintage Store,Gift Shop,Diner,Sandwich Place,Caribbean Restaurant,Chiropractor,Restaurant,Cocktail Bar
16,"India Bazaar, The Beaches West",Sandwich Place,Fast Food Restaurant,Park,Sushi Restaurant,Pub,Coffee Shop,Italian Restaurant,Movie Theater,Steakhouse,Food & Drink Shop
18,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Pharmacy,Thai Restaurant,Comfort Food Restaurant,Pizza Place,Pub,Restaurant,Café
19,Willowdale M2N,Ramen Restaurant,Coffee Shop,Korean Restaurant,Park,Sandwich Place,Pizza Place,Fast Food Restaurant,Sushi Restaurant,Café,Plaza
21,"The Annex, North Midtown, Yorkville",Sandwich Place,Café,Pharmacy,Italian Restaurant,Cosmetics Shop,Pizza Place,Donut Shop,Pub,Modern European Restaurant,Coffee Shop
22,"Parkdale, Roncesvalles",Coffee Shop,Grocery Store,Eastern European Restaurant,Sushi Restaurant,Restaurant,Thai Restaurant,Breakfast Spot,Gift Shop,Park,Bookstore
24,Davisville,Dessert Shop,Café,Sandwich Place,Coffee Shop,Pizza Place,Italian Restaurant,Park,Seafood Restaurant,Restaurant,Sushi Restaurant


#### Cluster 4

In [385]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Lawrence Manor, Lawrence Heights",Clothing Store,Toy / Game Store,American Restaurant,Furniture / Home Store,Men's Store,Bookstore,Restaurant,Cosmetics Shop,Women's Store,Coffee Shop
11,"Fairview, Henry Farm, Oriole",Clothing Store,Restaurant,Coffee Shop,Fast Food Restaurant,Food Court,Women's Store,Toy / Game Store,Bank,Boutique,Juice Bar


#### Cluster 5

In [386]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Queen's Park, Ontario Provincial Government",Coffee Shop,Café,Sushi Restaurant,Yoga Studio,Park,Spa,Burger Joint,Burrito Place,Sandwich Place,College Auditorium
